# **Stock Investment Recommendation Model**

## **Business Overview for Stock Investment Recommendation Model**

---

**Project Title:** Stock Investment Recommendation System

**Overview:**  
This project aims to create a machine learning model that delivers personalized stock investment recommendations based on individual risk appetites. By analyzing historical data from Yahoo Finance, the system will suggest suitable investments across various sectors, empowering investors with data-driven insights to enhance their decision-making and improve investment outcomes.

In [1]:
#  importing libraries
import pandas as pd
import numpy as np

In [2]:
import yfinance as yf
from datetime import datetime # Ensure pandas is imported

# List of stock symbols for 30 companies across different sectors
stock_symbols = [
    "AAPL", "XOM", "JPM", "PG", "JNJ", "NEE", "MSFT", "AMZN", "TSLA", "GOOGL",
    "NVDA", "V", "MA", "DIS", "NFLX", "INTC", "CSCO", "PFE", "KO", "PEP",
    "T", "VZ", "CMCSA", "ADBE", "IBM", "NKE", "CRM", "LLY", "ABT", "MDT"
]

# Define the time period for historical data
start_date = "2022-01-01"
end_date = datetime.today().strftime('%Y-%m-%d')

# creating benchmark
benchmark_symbol = "^GSPC"
benchmark_data = yf.download(benchmark_symbol, start=start_date, end=end_date)
benchmark_data['Benchmark_Return'] = benchmark_data['Adj Close'].pct_change()

# Create an empty list to store each DataFrame
data_frames = []

# Download historical stock data for each company and append to the list
for symbol in stock_symbols:
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)  # Reset index to make 'Date' a column
    stock_data['Symbol'] = symbol  # Add a 'Symbol' column for identification
    stock_data['Stock_Return'] = stock_data['Adj Close'].pct_change() 
    data_frames.append(stock_data)  # Add each DataFrame to the list

# Concatenate all DataFrames along rows (axis=0)
all_stocks_data = pd.concat(data_frames, axis=0, ignore_index=True)

# Merge stock data with benchmark data on Date to align the returns
combined_data = pd.merge(
    all_stocks_data[['Date', 'Symbol', 'Stock_Return']],
    benchmark_data[['Benchmark_Return']],
    left_on='Date', right_index=True, how='inner'
)
# Calculate beta for each stock and add it as a column
combined_data['Beta'] = np.nan # Initialize the Beta column with NaN values

# Loop through each stock to calculate and assign beta values
for symbol in stock_symbols:
    # Filter data for the specific stock
    stock_data = combined_data[combined_data['Symbol'] == symbol]
    
    # Calculate beta: Cov(Stock_Return, Benchmark_Return) / Var(Benchmark_Return)
    covariance = np.cov(stock_data['Stock_Return'].dropna(), stock_data['Benchmark_Return'].dropna())[0][1]
    benchmark_variance = np.var(stock_data['Benchmark_Return'].dropna())
    beta = covariance / benchmark_variance
    
    # Assign beta to the rows corresponding to the current stock symbol
    combined_data.loc[combined_data['Symbol'] == symbol, 'Beta'] = beta

# Merge the Beta values back into all_stocks_data based on Date and Symbol
all_stocks_data = pd.merge(
    all_stocks_data, 
    combined_data[['Date', 'Symbol', 'Beta']], 
    on=['Date', 'Symbol'], 
    how='left'
)


# Check if the DataFrame is structured correctly
print("Combined DataFrame columns:", all_stocks_data.columns)
print("Combined DataFrame sample:\n", all_stocks_data.head())

# Now pivot the DataFrame to have price types as columns
# Make sure to drop any NaN values before pivoting if necessary
pivot_stocks_data = all_stocks_data.pivot_table(
    index=['Date', 'Symbol'], 
    values=['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Beta'], 
    aggfunc='first'  # Use 'first' to handle potential duplicates
).reset_index()

# Display the first few rows of the pivoted DataFrame
print("Pivoted DataFrame sample:\n", pivot_stocks_data.head())

# Save the DataFrame to a CSV file
csv_file_path = "stocks_data.csv"  
pivot_stocks_data.to_csv(csv_file_path, index=False) 


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Combined DataFrame columns: MultiIndex([(        'Date',     ''),
            (   'Adj Close', 'AAPL'),
            (       'Close', 'AAPL'),
            (        'High', 'AAPL'),
            (         'Low', 'AAPL'),
            (        'Open', 'AAPL'),
            (      'Volume', 'AAPL'),
            (      'Symbol',     ''),
            ('Stock_Return',     ''),
            (   'Adj Close',  'XOM'),
            ...
            (         'Low',  'ABT'),
            (        'Open',  'ABT'),
            (      'Volume',  'ABT'),
            (   'Adj Close',  'MDT'),
            (       'Close',  'MDT'),
            (        'High',  'MDT'),
            (         'Low',  'MDT'),
            (        'Open',  'MDT'),
            (      'Volume',  'MDT'),
            (        'Beta',     '')],
           names=['Price', 'Ticker'], length=184)
Combined DataFrame sample:
 Price                       Date   Adj Close       Close        High  \
Ticker                                  AAPL 

C:\Users\DELL\AppData\Local\Temp\ipykernel_15004\1659082114.py:71: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  pivot_stocks_data = all_stocks_data.pivot_table(


Pivoted DataFrame sample:
 Price                       Date Symbol   Adj Close                          \
Ticker                                         AAPL         ABT        ADBE   
0      2022-01-03 00:00:00+00:00   AAPL  179.273621         NaN         NaN   
1      2022-01-03 00:00:00+00:00    ABT         NaN  131.440399         NaN   
2      2022-01-03 00:00:00+00:00   ADBE         NaN         NaN  564.369995   
3      2022-01-03 00:00:00+00:00   AMZN         NaN         NaN         NaN   
4      2022-01-03 00:00:00+00:00  CMCSA         NaN         NaN         NaN   

Price                                       ... Volume                       \
Ticker        AMZN      CMCSA CRM CSCO DIS  ...    NKE NVDA PEP PFE  PG   T   
0              NaN        NaN NaN  NaN NaN  ...    NaN  NaN NaN NaN NaN NaN   
1              NaN        NaN NaN  NaN NaN  ...    NaN  NaN NaN NaN NaN NaN   
2              NaN        NaN NaN  NaN NaN  ...    NaN  NaN NaN NaN NaN NaN   
3       170.404495      

In [3]:
df=pd.read_csv('stocks_data.csv')
df.head(15)

C:\Users\DELL\AppData\Local\Temp\ipykernel_15004\537850898.py:1: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('stocks_data.csv')


,Date,Symbol,Adj Close,Adj Close.1,Adj Close.2,Adj Close.3,Adj Close.4,Adj Close.5,Adj Close.6,Adj Close.7,...,Volume.20,Volume.21,Volume.22,Volume.23,Volume.24,Volume.25,Volume.26,Volume.27,Volume.28,Volume.29
0,NaN,NaN,AAPL,ABT,ADBE,AMZN,CMCSA,CRM,CSCO,DIS,...,NKE,NVDA,PEP,PFE,PG,T,TSLA,V,VZ,XOM
1,2022-01-03 00:00:00+00:00,AAPL,179.27362060546875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-03 00:00:00+00:00,ABT,NaN,131.44039916992188,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-03 00:00:00+00:00,ADBE,NaN,NaN,564.3699951171875,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-03 00:00:00+00:00,AMZN,NaN,NaN,NaN,170.4044952392578,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2022-01-03 00:00:00+00:00,CMCSA,NaN,NaN,NaN,NaN,46.967918395996094,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2022-01-03 00:00:00+00:00,CRM,NaN,NaN,NaN,NaN,NaN,254.3309326171875,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2022-01-03 00:00:00+00:00,CSCO,NaN,NaN,NaN,NaN,NaN,NaN,57.983245849609375,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2022-01-03 00:00:00+00:00,DIS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155.53321838378906,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2022-01-03 00:00:00+00:00,GOOGL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Step 1: Melt the DataFrame to long format
melted_df = pd.melt(df, id_vars=['Date', 'Symbol'], 
                    value_vars=[col for col in df.columns if col not in ['Date', 'Symbol']],
                    var_name='Variable', value_name='Value')

# Step 2: Extract stock type and suffix
melted_df[['Type', 'Stock']] = melted_df['Variable'].str.split('.', expand=True)

# Step 3: Pivot to get the desired structure
final_df = melted_df.pivot_table(index=['Date', 'Symbol'], columns=['Type'], values='Value', aggfunc='first')

# Step 4: Flatten MultiIndex columns
final_df.columns = [col.strip() for col in final_df.columns.values]
final_df.reset_index(inplace=True)

# Step 5: Ensure desired columns are present
desired_columns = ['Date', 'Symbol', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume','Beta']
# Select only the columns present in the DataFrame to avoid KeyError
final_df = final_df[[col for col in desired_columns if col in final_df.columns]]

# Step 6: Convert 'Date' to datetime and sort
final_df['Date'] = pd.to_datetime(final_df['Date'])
final_df.sort_values(by='Date', inplace=True)

# Step 7: Reset index after sorting
final_df.reset_index(drop=True, inplace=True)

# Display the first few rows of the final DataFrame
print(final_df)

# Optionally save to CSV if needed
final_df.to_csv('final_stocks_data.csv', index=False)

                           Date Symbol                Open  \
0     2022-01-03 00:00:00+00:00   AAPL   177.8300018310547   
1     2022-01-03 00:00:00+00:00    XOM    61.2400016784668   
2     2022-01-03 00:00:00+00:00     VZ   52.06999969482422   
3     2022-01-03 00:00:00+00:00      V  217.52000427246094   
4     2022-01-03 00:00:00+00:00   TSLA   382.5833435058594   
...                         ...    ...                 ...   
21505 2024-11-07 00:00:00+00:00    ABT          117.489998   
21506 2024-11-07 00:00:00+00:00   AAPL          224.630005   
21507 2024-11-07 00:00:00+00:00     VZ           41.080002   
21508 2024-11-07 00:00:00+00:00     KO           64.290001   
21509 2024-11-07 00:00:00+00:00    XOM          121.300003   

                     High                 Low               Close  \
0       182.8800048828125   177.7100067138672  182.00999450683594   
1      63.599998474121094  61.209999084472656  63.540000915527344   
2      52.560001373291016   51.97999954223633  5